# Импорт необходимых библиотек

In [1]:
import numpy as np
import pandas as pd
import pylab as pl
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
import seaborn as sns

### Считываем данные из файла, пустые значения заменяем на медианные

In [3]:
df=pd.read_csv(r"C:\Users\Rasim\Desktop\DataAnalysis\DataSets\ORGANICS.csv",encoding="cp1251")
print(df.shape) 
print(df.info())
df.fillna(df.median(axis=0), axis=0, inplace=True) 
df

(22223, 13)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22223 entries, 0 to 22222
Data columns (total 13 columns):
ID                 22223 non-null int64
DemAffl            21138 non-null float64
DemAge             20715 non-null float64
DemCluster         21549 non-null float64
DemClusterGroup    21549 non-null object
DemGender          19711 non-null object
DemReg             21758 non-null object
DemTVReg           21758 non-null object
PromClass          22223 non-null object
PromSpend          22223 non-null float64
PromTime           21942 non-null float64
TargetBuy          22223 non-null int64
TargetAmt          22223 non-null int64
dtypes: float64(5), int64(3), object(5)
memory usage: 2.2+ MB
None


,ID,DemAffl,DemAge,DemCluster,DemClusterGroup,DemGender,DemReg,DemTVReg,PromClass,PromSpend,PromTime,TargetBuy,TargetAmt
0,140,10.0,76.0,16.0,C,U,Midlands,Wales & West,Gold,16000.00,4.0,0,0
1,620,4.0,49.0,35.0,D,U,Midlands,Wales & West,Gold,6000.00,5.0,0,0
2,868,5.0,70.0,27.0,D,F,Midlands,Wales & West,Silver,0.02,8.0,1,1
3,1120,10.0,65.0,51.0,F,M,Midlands,Midlands,Tin,0.01,7.0,1,1
4,2313,11.0,68.0,4.0,A,F,Midlands,Midlands,Tin,0.01,8.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22218,52834058,13.0,65.0,9.0,B,F,South East,London,Silver,1500.00,5.0,0,0
22219,52834376,15.0,73.0,34.0,D,U,South East,S & S East,Gold,6053.06,12.0,0,0
22220,52837057,9.0,70.0,15.0,B,F,North,Yorkshire,Gold,6000.00,5.0,0,0
22221,52838096,11.0,66.0,8.0,B,F,North,N West,Silver,5000.00,5.0,0,0


#### Для анализа в этой части используем пока только количественные признаки 

In [5]:
x=df[['DemAffl',  'DemAge', 'DemCluster', 'PromSpend', 'PromTime']]
y=df.values[::,11].astype('float64')


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

### Обучение дерева
### (max_depth=5, criterion=entropy)

In [6]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth=5, random_state=0, criterion='entropy')
tree.fit(X_train, y_train)


DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=0, splitter='best')

In [7]:
print("Результат на тестовой выборке:", tree.score(X_test, y_test))
print("Результат на тренировочной выборке:", tree.score(X_train, y_train))

Результат на тестовой выборке: 0.80005999700015
Результат на тренировочной выборке: 0.8013628182051942


### Поиск наилучшего дерева путем перебора комбинаций max_depth, criterion, min_samples_leaf

In [8]:
data = pd.DataFrame(columns=['max_depth', 'criterion', 'min_samples_leaf', 'train_result', 'test_result'])
for criterion in ['entropy', 'gini']:
    for max_depth in range(5, 100):
        for min_leaf in range(1, 20):
            tree = DecisionTreeClassifier(max_depth=max_depth, random_state=0, criterion=criterion, min_samples_leaf=min_leaf)
            tree.fit(X_train, y_train)
            train_result = tree.score(X_train, y_train)
            test_result = tree.score(X_test, y_test)
            data = data.append({'max_depth': max_depth,
                         'criterion': criterion, 
                         'min_samples_leaf': min_leaf,
                         'train_result': train_result,
                         'test_result': test_result}, ignore_index=True)


#### Найдем в этой таблице дерево с наибольшим результатом на тестовой выборке

In [9]:
index = np.argmax(np.array(data['test_result']))
data.iloc[index]

max_depth                 7
criterion           entropy
min_samples_leaf          7
train_result        0.80657
test_result         0.80411
Name: 44, dtype: object

И так, параметры наилучшего дерева для тестовой выборки(точность - 80,7%):
- max_depth: 7
- criterion: entropy
- min_samples_leaf: 11

### Визуализация дерева

In [12]:
from sklearn.tree import export_graphviz
import pydot


best_tree = DecisionTreeClassifier(max_depth=7, random_state=0, criterion='entropy', min_samples_leaf=11)
best_tree.fit(X_train, y_train)
export_graphviz(best_tree, out_file='Organics_tree.dot', rounded = True, proportion = False, precision = 2, filled = True)
(graph, ) = pydot.graph_from_dot_file('Organics_tree.dot')

graph.write_png('Organics_tree.png')


def get_grid(data):
    x_min, x_max = data[:, 0].min() - 1, data[:, 0].max() + 1
    y_min, y_max = data[:, 1].min() - 1, data[:, 1].max() + 1
    return np.meshgrid(np.arange(x_min, x_max, 0.01),
                         np.arange(y_min, y_max, 0.01))


plt.scatter(train_data[:, 0], train_data[:, 1], c=train_labels, s=100, cmap='autumn')


#### Найдем в этой таблице дерево с наибольшим результатом на тренировочной выборке, естественно, оно будет перетренировано

In [40]:
index = np.argmax(np.array(data['train_result']))
data.iloc[index]

max_depth                 41
criterion            entropy
min_samples_leaf           1
train_result        0.995629
test_result         0.712764
Name: 684, dtype: object

И так, параметры наилучшего дерева для тренировочной выборки(точность - 99,6%):
    - max_depth: 41
    - criterion: entropy
    - min_samples_leaf: 1